<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/MegaByte_pytroch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

base: 基础普适研究

- hourglass transformers: [2021. Hierarchical Transformers Are More Efficient Language Models](https://arxiv.org/abs/2110.13711) | [lucidrains/simple-hierarchical-transformer](https://github.com/lucidrains/simple-hierarchical-transformer) vanilla layers and shortened layers use GPT  AR  GLM 🤞
- [2023.5 **MEGABYTE: Predicting Million-byte Sequences with Multiscale Transformers**](https://arxiv.org/abs/2305.07185) | [lucidrains/MEGABYTE-pytorch](https://github.com/lucidrains/MEGABYTE-pytorch)

PS: 想法和自己整理的[simple LM](https://github.com/ai-bot-pro/baby-llm/tree/main/simpleLM)相似~

# hourglass transformer

hourglass transformers: [2021. Hierarchical Transformers Are More Efficient Language Models](https://arxiv.org/abs/2110.13711) | [lucidrains/simple-hierarchical-transformer](https://github.com/lucidrains/simple-hierarchical-transformer) vanilla layers and shortened layers use GPT  AR  GLM 🤞


In [1]:
!git clone https://github.com/lucidrains/simple-hierarchical-transformer.git

Cloning into 'simple-hierarchical-transformer'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 312 (delta 68), reused 69 (delta 68), pack-reused 241 (from 1)
Receiving objects: 100% (312/312), 34.92 MiB | 24.52 MiB/s, done.
Resolving deltas: 100% (224/224), done.


In [5]:
%cd /content/simple-hierarchical-transformer

/content/simple-hierarchical-transformer


In [ ]:
!pip install -e .

In [7]:
import torch
from simple_hierarchical_transformer import HierarchicalTransformer

model = HierarchicalTransformer(
    num_tokens = 20000,                # number of tokens
    dim = 512,                         # model dimensions
    depth = 6,                         # depth
    dim_head = 64,                     # dimension per attention head
    heads = 8,                         # attention heads
    seq_len = 2048,                    # sequence lengths
    hierarchies = (1, 2, 8),           # hierarchies - here we have 1x (like in a regular transformer), then 2x and 8x compressed hierarchical tokens that undergo their own transformer blocks. information is pooled into one hierarchy at each layer
    window_sizes = (32, 64, None)      # local attention window sizes - the idea is that the higher hierarchies can pass distant information to the local one. None stands for full receptive field. Setting 0 would turn off attention at this hierarchy altogether (while token shift will still be in effect in each layer)
)
print(model)

ids = torch.randint(0, 20000, (1, 2048))
print(ids.shpae)
loss, _ = model(ids, return_loss = True)
print(loss)
loss.backward()

# after much training

logits = model(ids)
print(logits)

In [8]:
# non-hierarchical transformer

model = HierarchicalTransformer(
    num_tokens = 20000,
    dim = 512,
    depth = 8,
    dim_head = 64,
    heads = 8,
    seq_len = 2048,
    hierarchies = 1,        # implied 1 if not set
    window_sizes = None     # implied None (full sequence length) if not set
)
print(model)

HierarchicalTransformer(
  (token_emb): Embedding(20000, 512)
  (to_hierarchical_preds): ModuleList(
    (0): None
  )
  (compressors): ModuleList(
    (0): Compress(
      (compress_fn): Identity()
    )
  )
  (post_token_emb_norms): ModuleList(
    (0): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (layers): ModuleList(
    (0-7): 8 x ModuleList(
      (0): HierarchicalBlock(
        (attn): Attention(
          (norm): RMSNorm()
          (rotary_emb): RotaryEmbedding()
          (attend): Attend()
          (to_qkv): Linear(in_features=512, out_features=1536, bias=False)
          (to_out): Linear(in_features=512, out_features=512, bias=False)
        )
        (ff): FeedForward(
          (net): Sequential(
            (0): RMSNorm()
            (1): Linear(in_features=512, out_features=2048, bias=False)
            (2): GELU(approximate='none')
            (3): Linear(in_features=2048, out_features=512, bias=False)
          )
        )
      )
    )
  )
  (hierarch

In [9]:
model = HierarchicalTransformer(
    num_tokens = 256,
    dim = (128, 256, 512, 1024),
    depth = 8,
    seq_len = 1024,
    use_flash_attn = True,
    ff_mult = (2, 2, 4, 4),
    dim_head = (16, 32, 64, 64),
    heads = (2, 4, 8, 8),
    hierarchies = (1, 2, 4, 16),
    hierarchical_stride = (1, 1, 1, 8),  # this would determine the stride when compressing, and when concatting the hierarchical tokens to the fine tokens, the past tokens will be repeated this amount of time. causality is not violated as using the trick from hourglass transformers where sequence is shifted by compression factor - 1. recommend sticking with 1 except for highly compressed hierarchies, as it becomes very uncompetitive with baseline and generations look off
    window_sizes = (16, 32, 64, None)
).cuda()

print(model)

# hierarchies
# 1x - dim 128 - attention (2 heads, 16 dim, receptive field 16)
# 2x - dim 256 - attention (4 heads, 32 dim, receptive field 32)
# 4x - dim 512 - attention (8 heads, 64 dim, receptive field 64)
# 8x - dim 1024 - attention (8 heads, 64 dim, receptive field of all)

HierarchicalTransformer(
  (token_emb): Embedding(256, 1024)
  (to_hierarchical_preds): ModuleList(
    (0): None
    (1): Linear(in_features=256, out_features=256, bias=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (compressors): ModuleList(
    (0): Compress(
      (compress_fn): Linear(in_features=1024, out_features=128, bias=False)
    )
    (1): Compress(
      (compress_fn): Sequential(
        (0): Rearrange('b n d -> b d n')
        (1): CausalConv(
          (conv): Conv1d(1024, 4096, kernel_size=(2,), stride=(1,))
        )
        (2): SiLU()
        (3): Conv1d(4096, 256, kernel_size=(1,), stride=(1,))
        (4): Rearrange('b d n -> b n d')
      )
      (to_recon): Linear(in_features=256, out_features=512, bias=False)
    )
    (2): Compress(
      (compress_fn): Sequential(
        (0): Rearrange('b n d -> b d n')
        (1): CausalConv(
          (conv): Conv1d(1024, 4096, kern

In [10]:
!python train.py


/content/simple-hierarchical-transformer/simple_hierarchical_transformer/simple_hierarchical_transformer.py:109: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/content/simple-hierarchical-transformer/simple_hierarchical_transformer/simple_hierarchical_transformer.py:138: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
training:   0% 0/100000 [00:00<?, ?it/s]training loss: 5.7180633544921875
validation loss: 4.1848015785217285
%s 

 %s ('rthodox iconography in the interpretation of [[Simon Ushakov]] (1658).]]  In addition to the legend that Pilate had made an imag', '****************************************************************************************************')
   t  a  e te   t o   o      or    o th  ho   |hM   s|   a  ietit t  [ oPi   [  or th1 e tz e       e      l    h 

# MEGABYTE-pytorch

In [12]:
%cd /content

/content


In [14]:
!git clone https://github.com/lucidrains/MEGABYTE-pytorch.git

Cloning into 'MEGABYTE-pytorch'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 188 (delta 98), reused 77 (delta 76), pack-reused 62 (from 1)
Receiving objects: 100% (188/188), 35.31 MiB | 25.92 MiB/s, done.
Resolving deltas: 100% (126/126), done.


In [2]:
%cd /content/MEGABYTE-pytorch

/content/MEGABYTE-pytorch


In [ ]:
!pip install -e .

In [ ]:
import torch
from MEGABYTE_pytorch import MEGABYTE

model = MEGABYTE(
    num_tokens = 16000,             # number of tokens
    dim = (512, 256),               # transformer model dimension (512 for coarsest, 256 for fine in this example)
    max_seq_len = (1024, 4),        # sequence length for global and then local. this can be more than 2
    depth = (6, 4),                 # number of layers for global and then local. this can be more than 2, but length must match the max_seq_len's
    dim_head = 64,                  # dimension per head
    heads = 8,                      # number of attention heads
    flash_attn = True               # use flash attention
)
print(model)

x = torch.randint(0, 16000, (1, 1024, 4))
print(x.shape)

loss = model(x, return_loss = True)
print(loss)
loss.backward()

# then after much training

logits = model(x)
print(logits)

# and sample from the logits accordingly
# or you can use the generate function

sampled = model.generate(temperature = 0.9, filter_thres = 0.9) # (1, 1024, 4)
print(sampled)

Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda
MEGABYTE(
  (start_tokens): ParameterList(
      (0): Parameter containing: [torch.float32 of size 512]
      (1): Parameter containing: [torch.float32 of size 256]
  )
  (token_embs): ModuleList(
    (0): Embedding(16000, 256)
    (1): Sequential(
      (0): Embedding(16000, 256)
      (1): Rearrange('... r d -> ... (r d)')
      (2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (3): Linear(in_features=1024, out_features=512, bias=True)
      (4): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
  )
  (transformers): ModuleList(
    (0): Transformer(
      (layers): ModuleList(
        (0-5): 6 x ModuleList(
          (0): Attention(
            (attend): Attend(
              (attn_dropout): Dropout(p=0.0, inplace=False)
            )
            (dropout): Dropout(p=0.0, inplace=False)
            (norm): RMSNorm()
            (to_q): Linear(in_features=512, out_feature

 61%|██████▏   | 2514/4096 [33:14<1:08:40,  2.60s/it]

In [3]:
!python train.py

Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda
training:   0% 0/100000 [00:00<?, ?it/s]training loss: 5.705471515655518
validation loss: 5.229538440704346
training loss: 5.157638072967529
training loss: 4.804471969604492
training:   0% 3/100000 [00:10<100:40:18,  3.62s/it]training loss: 4.644378185272217
training loss: 4.377236366271973
training loss: 4.3670196533203125
training loss: 4.2122907638549805
training:   0% 7/100000 [00:23<90:50:02,  3.27s/it] training loss: 4.318767547607422
training loss: 4.108756065368652
training loss: 4.0011067390441895
training loss: 3.890439748764038
training:   0% 11/100000 [00:35<88:44:14,  3.19s/it]training loss: 3.8987667560577393
training loss: 3.9163074493408203
training loss: 3.841243267059326
training loss: 3.8642024993896484
training:   0% 15/100000 [00:48<88:26:14,  3.18s/it]training loss: 3.62235164642334
training loss: 3.9597721099853516
training loss: 3.819218158721924
training:   0% 15/100000 [01: